In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession\
    .builder\
    .appName("twitter_silver")\
    .getOrCreate()


23/09/25 22:16:32 WARN Utils: Your hostname, Estudos resolves to a loopback address: 127.0.1.1; using 192.168.0.9 instead (on interface enp3s0)
23/09/25 22:16:32 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/25 22:16:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
df_tweet = spark.read.json("/home/marcelo/Documents/airflow/datalake/silver/twitter_datascience/tweet")

In [5]:
df_tweet.printSchema()

root
 |-- author_id: string (nullable = true)
 |-- conversation_id: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- id: string (nullable = true)
 |-- like_count: long (nullable = true)
 |-- quote_count: long (nullable = true)
 |-- reply_count: long (nullable = true)
 |-- retweet_count: long (nullable = true)
 |-- text: string (nullable = true)
 |-- process_date: date (nullable = true)



In [6]:
from pyspark.sql import functions as f

In [7]:
tweet_conversas = df_tweet.alias("tweet")\
                  .groupBy(f.to_date("created_at").alias("created_date"))\
                    .agg(
                        f.countDistinct("author_id").alias("n_tweets"),
                        f.sum("like_count").alias("n_like"),
                        f.sum("quote_count").alias("n_quote"),
                        f.sum("reply_count").alias("n_reply"),
                        f.sum("retweet_count").alias("n_retweet"),
                    ).withColumn("weekday", f.date_format("created_date", "E"))

In [8]:
tweet_conversas.show()

+------------+--------+------+-------+-------+---------+-------+
|created_date|n_tweets|n_like|n_quote|n_reply|n_retweet|weekday|
+------------+--------+------+-------+-------+---------+-------+
|  2023-09-25|       9|   948|    798|    928|      970|    Mon|
|  2023-09-24|      23|  3490|   2694|   3430|     2958|    Sun|
+------------+--------+------+-------+-------+---------+-------+

